# Vectorizing Raw Data: Count Vectorization

### Count vectorization 

Creates a document-term matrix where the entry of each cell will be a count of the number of times that word occurred in that document.

### Read in text

In [9]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

data.head()

,label,body_text
0,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
1,ham,"Nah I don't think he goes to usf, he lives around here though"
2,ham,Even my brother is not like to speak with me. They treat me like aids patent.
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
4,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your call...


### Create function to remove punctuation, tokenize, remove stopwords, and stem

In [10]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

### Apply CountVectorizer

We are not using lambda function to apply `clean_text` function to out data.
Instead we're using `CountVectorizer` from sklearn, which allows to pass in a function to clean and tokenize your data.

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer=clean_text)
# X_counts = count_vect.fit(data['body_text'])
# If we only `fit` it, it won't do anything to our data, it will just train the vectorizer to learn what words are in the corpus
# to actually fit it and then transform our data (fit the vectorizer and vectorize our data and turn them into feature vectors), 
# we need to call `fit_transform`:
X_counts = count_vect.fit_transform(data['body_text'])

X_counts

<5567x8104 sparse matrix of type '<class 'numpy.int64'>'
	with 50122 stored elements in Compressed Sparse Row format>

In [33]:
print(X_counts.shape)
# rows - text messages; columns - unique words;
print(count_vect.get_feature_names()[-100:])
# the names of the columns of our document-term matrix

(5567, 8104)
['yeah', 'yeahand', 'year', 'yeesh', 'yeh', 'yell', 'yellow', 'yelowi', 'yen', 'yeovil', 'yep', 'yer', 'yes165', 'yes434', 'yes440', 'yes762', 'yes910', 'yesbut', 'yesfrom', 'yesgauti', 'yesh', 'yesher', 'yesim', 'yesmum', 'yessura', 'yest', 'yesterday', 'yet', 'yetti', 'yetund', 'yi', 'yifeng', 'yiju', 'yijuehotmailcom', 'ym', 'ymca', 'yo', 'yoga', 'yogasana', 'yoher', 'yor', 'yorg', 'youani', 'youcarlo', 'youclean', 'youd', 'youdearwith', 'youdo', 'youhow', 'youi', 'youkwher', 'yould', 'youll', 'youmi', 'youmoney', 'young', 'younger', 'youphon', 'your', 'yourinclus', 'yourjob', 'youso', 'youthat', 'youto', 'youuuuu', 'youv', 'youwanna', 'youwhen', 'yovil', 'yowif', 'yoyyooo', 'yr', 'ystrdayic', 'yummi', 'yummmm', 'yun', 'yunni', 'yuo', 'yuou', 'yup', 'yupz', 'ywhere', 'z', 'zac', 'zaher', 'zealand', 'zebra', 'zed', 'zero', 'zhong', 'zindgi', 'zoe', 'zogtoriu', 'zoom', 'zouk', 'zyada', 'é', 'ü', 'üll', '〨ud']


### Apply CountVectorizer to smaller sample

In [34]:
data_sample = data[0:20]

count_vect_sample = CountVectorizer(analyzer=clean_text)
X_counts_sample = count_vect_sample.fit_transform(data_sample['body_text'])
print(X_counts_sample.shape)
print(count_vect_sample.get_feature_names())

(20, 192)
['08002986030', '08452810075over18', '09061701461', '1', '100', '100000', '11', '12', '150pday', '16', '2', '20000', '2005', '21st', '3', '4', '4403ldnw1a7rw18', '4txtú120', '6day', '81010', '87077', '87121', '87575', '9', '900', 'aft', 'aid', 'alreadi', 'alright', 'anymor', 'appli', 'ard', 'around', 'b', 'brother', 'call', 'caller', 'callertun', 'camera', 'cash', 'chanc', 'claim', 'click', 'co', 'code', 'colour', 'comin', 'comp', 'copi', 'cost', 'credit', 'cri', 'csh11', 'cup', 'custom', 'da', 'date', 'dont', 'eg', 'eh', 'england', 'enough', 'entitl', 'entri', 'even', 'fa', 'feel', 'ffffffffff', 'final', 'fine', 'finish', 'first', 'free', 'friend', 'go', 'goalsteam', 'goe', 'gonna', 'gota', 'ha', 'hl', 'home', 'hour', 'httpwap', 'im', 'info', 'ive', 'jackpot', 'joke', 'k', 'kim', 'kl341', 'lar', 'latest', 'lccltd', 'like', 'link', 'live', 'lor', 'lunch', 'macedonia', 'make', 'may', 'meet', 'mell', 'membership', 'messag', 'minnaminungint', 'miss', 'mobil', 'month', 'nah', 'na

### Vectorizers output sparse matrices

_**Sparse Matrix**: A matrix in which most entries are 0. In the interest of efficient storage, a sparse matrix will be stored by only storing the locations of the non-zero elements._

In [35]:
X_counts_sample

<20x192 sparse matrix of type '<class 'numpy.int64'>'
	with 218 stored elements in Compressed Sparse Row format>

In [36]:
X_counts_df = pd.DataFrame(X_counts_sample.toarray())
# toarray - for each row, instead of storing just the non-zero elements, it adds those zeroes back in there,
# creating for each row a full array that is 192 entries long

X_counts_df

,0,1,2,3,4,5,6,7,8,9,...,182,183,184,185,186,187,188,189,190,191
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
6,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,1,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
9,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [37]:
X_counts_df.columns = count_vect_sample.get_feature_names()
X_counts_df

,08002986030,08452810075over18,09061701461,1,100,100000,11,12,150pday,16,...,wet,win,winner,wkli,word,wwwdbuknet,xxxmobilemovieclub,xxxmobilemovieclubcomnqjkgighjjgcbl,ye,ü
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
6,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,1,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
9,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
